[IPython Notebook](alternate_backends.ipynb) |  [Python Script](alternate_backends.py)

Advanced: Alternate Backends
============================

Setup
-----------------------------

As always, let's do imports and initialize a logger and a new Bundle.  See [Building a System](building_a_system.html) for more details.

In [1]:
import phoebe
from phoebe import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe.logger()

b = phoebe.default_binary()

And we'll attach some dummy datasets.  See [Datasets](datasets.html) for more details.

In [2]:
b.add_dataset('orb', times=np.linspace(0,10,1000), dataset='orb01', component=['primary', 'secondary'])

<ParameterSet: 3 parameters | contexts: compute, dataset>

In [3]:
b.add_dataset('lc', times=np.linspace(0,10,1000), dataset='lc01')

<ParameterSet: 15 parameters | contexts: compute, dataset>

Available Backends
----------------------------

See the [Compute Tutorial](./compute) for details on adding compute options and using them to create synthetic models.


### PHOEBE 1.0 Legacy

For more details, see [Comparing PHOEBE 2.0 vs PHOEBE Legacy](../examples/legacy)

In [4]:
b.add_compute('legacy', compute='legacybackend')
print b['legacybackend']

ParameterSet: 7 parameters
  refl_num@legacybackend@compute: 1
        ie@legacybackend@compute: False
  enabled@lc01@legacybackend@...: True
  atm@primary@legacybackend@c...: extern_atmx
  atm@secondary@legacybackend...: extern_atmx
  gridsize@primary@legacyback...: 60
  gridsize@secondary@legacyba...: 60


Using Alternate Backends
---------------------------------


### Adding Compute Options

Adding a set of compute options for an alternate backend is just as easy as for the PHOEBE backend.  Simply provide the function or name of the function in phoebe.parameters.compute that points to the parameters for that backend.

Here we'll add the default PHOEBE backend as well as the PHOEBE 1.0 (legacy) backend.  Note that in order to use an alternate backend, that backend must be installed on your machine.

In [5]:
b.add_compute('phoebe', compute='phoebebackend')

<ParameterSet: 18 parameters | datasets: lc01, orb01>

In [6]:
print b['phoebebackend']

ParameterSet: 18 parameters
  dynamics_method@phoebebacke...: keplerian
      ltte@phoebebackend@compute: False
  irrad_method@phoebebackend@...: wilson
  boosting_method@phoebebacke...: none
  eclipse_method@phoebebacken...: native
  horizon_method@phoebebacken...: boolean
  enabled@lc01@phoebebackend@...: True
  enabled@orb01@phoebebackend...: True
  mesh_method@primary@phoebeb...: marching
  mesh_method@secondary@phoeb...: marching
  ntriangles@primary@phoebeba...: 1500
  ntriangles@secondary@phoebe...: 1500
  distortion_method@primary@p...: roche
  distortion_method@secondary...: roche
  atm@primary@phoebebackend@c...: ck2004
  atm@secondary@phoebebackend...: ck2004
  lc_method@lc01@phoebebacken...: numerical
  fti_method@lc01@phoebebacke...: none


### Running Compute

Nothing changes for running compute - simply provide the compute tag for those options.  Do note, however, that not all backends support all dataset types.

But, since the legacy backend doesn't support ck2004 atmospheres and interpolated limb-darkening, we do need to choose a limb-darkening law.

In [7]:
b.set_value_all('ld_func', 'logarithmic')

Thu, 04 Oct 2018 14:48 PARAMETERS   WARNING ld_func='interp' only supported by atm='ck2004'.  Either change atm@primary or ld_func@primary@lc01  If not addressed, this warning will continue to be raised and will throw an error at run_compute.
Thu, 04 Oct 2018 14:48 PARAMETERS   WARNING ld_func='interp' only supported by atm='ck2004'.  Either change atm@secondary or ld_func@secondary@lc01  If not addressed, this warning will continue to be raised and will throw an error at run_compute.


In [8]:
b.run_compute('legacybackend', model='legacyresults')

Thu, 04 Oct 2018 14:48 IO           WARNING requiv_max@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:48 IO           WARNING freq@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:48 IO           WARNING pitch@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:48 IO           WARNING yaw@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:48 IO           WARNING long_an@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:48 IO           WARNING irrad_frac_lost_bol@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:48 IO           WARNING ld_func_bol@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:48 IO           WARNING mass@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:48 IO           WARNING requiv_max@secondary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:48 IO           WARNING freq@secondary@star@component has n

<ParameterSet: 2 parameters | qualifiers: fluxes, times>

Running Multiple Backends Simultaneously
---------------------------------------------------

Running multiple backends simultaneously is just as simple as running the PHOEBE backend with multiple sets of compute options (see [Compute](compute.html)).

We just need to make sure that each dataset is only enabled for one (or none) of the backends that we want to use, and then send a list of the compute tags to run_compute.  Here we'll use the PHOEBE backend to compute orbits and the legacy backend to compute light curves.

In [9]:
b.set_value_all('enabled@lc01@phoebebackend', False)
#b.set_value_all('enabled@orb01@legacybackend', False)  # don't need this since legacy NEVER computes orbits
print b['enabled']

ParameterSet: 5 parameters
  enabled@orb01@phoebe01@compute: True
   enabled@lc01@phoebe01@compute: True
  enabled@lc01@legacybackend@...: True
  enabled@lc01@phoebebackend@...: False
  enabled@orb01@phoebebackend...: True


In [10]:
b.run_compute(['phoebebackend', 'legacybackend'], model='mixedresults')

Thu, 04 Oct 2018 14:49 IO           WARNING requiv_max@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:49 IO           WARNING freq@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:49 IO           WARNING pitch@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:49 IO           WARNING yaw@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:49 IO           WARNING long_an@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:49 IO           WARNING irrad_frac_lost_bol@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:49 IO           WARNING ld_func_bol@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:49 IO           WARNING mass@primary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:49 IO           WARNING requiv_max@secondary@star@component has no phoebe 1 corollary
Thu, 04 Oct 2018 14:49 IO           WARNING freq@secondary@star@component has n

<ParameterSet: 16 parameters | kinds: orb, lc>

The parameters inside the returned model even remember which set of compute options (and therefore, in this case, which backend) were used to compute them.

In [11]:
print b['mixedresults'].computes

['legacybackend', 'phoebebackend']


In [12]:
b['mixedresults@phoebebackend'].datasets

['orb01']

In [13]:
b['mixedresults@legacybackend'].datasets

['lc01']